In [1]:
from google.cloud import vision
from google.oauth2 import service_account
import io
import re
import os

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="sturdy-quarter-361804-599639eb7da1.json"

In [3]:
bpkb_path = "image_sample/bpkb_pak_qwantes.jpeg"

In [4]:
client = vision.ImageAnnotatorClient()

In [5]:
with open(bpkb_path, "rb") as image_file:
    content = image_file.read()
    
image = vision.Image(content=content)

In [21]:
response = client.document_text_detection(
    image=image,
    image_context={"language_hints": ["mul-Latn-t-i0-handwrit"]}
)

In [22]:
response

text_annotations {
  locale: "id"
  description: "3\n1. Nomor Registrasi :8 4122 BNU\nVehicle Registration Number\nHONDA\nEIF02N12M2 A/T\n2. Merek\nBrand\n3. Type\nType\n4. Jenis\nCategory\n5. Model\nSEPEDA MOTOR\nSOLO\nModel\n6. Tahun Pembuatan :2017\nManufacture Year\n125 CC\n7. Isi Silinder\nCylinder Capacity\nMobil Penumpang\nPassenger Vehicle\nSedan\nSedan\nII. IDENTITAS KENDARAAN\nVEHICLE IDENTITY\n8. Warna\nHITAM\nColor\n9. Nomor Rangka/NIK/VIN:\nVehicle Identity Number\nMH1JFV119HK717286\nBukan Sedan\nNon-Sedan\nMobil Penumpang\nKeperluan Khusus\nPassenger Car for\nSpecific Purpose\nDODO\nMobil Bus\nBus Vehicle\nBus Kecil\nMicro Bus\nBus Besar\nLarge Bus\nBus Sedang\nMedium Bus\nMobil Barang\nCargo Vehicle\nBus Tingkat\nStorey Bus\n10. Nomor Mesin JFV1E1724636\nEngine Number\n11. Bahan Bakar BENSIN\nType Fuel\n12. Jumlah Sumbu\nNumber of Axis\n13. Jumlah Roda : 2 (DUA)\nNumber of Wheels\nMobil bak\nmuatan terbuka\nPick Up\nMobil bak\nmuatan tertutup\nDelivery Car\nBus Maxi\nMax

In [23]:
document = response.full_text_annotation
document

pages {
  property {
    detected_languages {
      language_code: "id"
      confidence: 0.45948657393455505
    }
    detected_languages {
      language_code: "en"
      confidence: 0.42955371737480164
    }
    detected_languages {
      language_code: "ms"
      confidence: 0.05500096082687378
    }
    detected_languages {
      language_code: "sv"
      confidence: 0.009499375708401203
    }
    detected_languages {
      language_code: "it"
      confidence: 0.008522148244082928
    }
  }
  width: 1600
  height: 900
  blocks {
    bounding_box {
      vertices {
        x: 82
        y: 823
      }
      vertices {
        x: 104
        y: 823
      }
      vertices {
        x: 104
        y: 849
      }
      vertices {
        x: 82
        y: 849
      }
    }
    paragraphs {
      bounding_box {
        vertices {
          x: 82
          y: 823
        }
        vertices {
          x: 104
          y: 823
        }
        vertices {
          x: 104
          y: 849


In [35]:
# import argparse
from enum import Enum
import io

from google.cloud import vision
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt


class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5


def draw_boxes(image, bounds, color):
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)

    for idx, bound in enumerate(bounds):
        draw.polygon(
            [
                bound.vertices[0].x,
                bound.vertices[0].y,
                bound.vertices[1].x,
                bound.vertices[1].y,
                bound.vertices[2].x,
                bound.vertices[2].y,
                bound.vertices[3].x,
                bound.vertices[3].y,
            ],
            None,
            color,
        )
        
        draw.text(
            (
                bound.vertices[0].x,
                bound.vertices[0].y - 10
            ), str(idx),
            color
        )
    return image


def get_document_bounds(image_file, feature):
    """Returns document bounds given an image."""
    client = vision.ImageAnnotatorClient()

    bounds = []

    with io.open(image_file, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(
        image=image,
        image_context={"language_hints": ["mul-Latn-t-i0-handwrit"]}
    )
    document = response.full_text_annotation

    # Collect specified feature bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if feature == FeatureType.SYMBOL:
                            bounds.append(symbol.bounding_box)

                    if feature == FeatureType.WORD:
                        bounds.append(word.bounding_box)

                if feature == FeatureType.PARA:
                    bounds.append(paragraph.bounding_box)

            if feature == FeatureType.BLOCK:
                bounds.append(block.bounding_box)

    # The list `bounds` contains the coordinates of the bounding boxes.
    return bounds


def render_doc_text(filein, fileout):
    image = Image.open(filein)
    bounds = get_document_bounds(filein, FeatureType.BLOCK)
    draw_boxes(image, bounds, "blue")
#     bounds = get_document_bounds(filein, FeatureType.PARA)
#     draw_boxes(image, bounds, "red")
#     bounds = get_document_bounds(filein, FeatureType.WORD)
#     draw_boxes(image, bounds, "yellow")

    if fileout != 0:
        image.save(fileout)
    else:
#         plt.imshow(image)
#         plt.axis("off")
#         plt.show()
        image.show()



# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument("detect_file", help="The image for text detection.")
#     parser.add_argument("-out_file", help="Optional output file", default=0)
#     args = parser.parse_args()

#     render_doc_text(args.detect_file, args.out_file)

In [36]:
render_doc_text(filein=bpkb_path, fileout=0)

In [26]:
len(document.pages)

1

In [27]:
len(document.pages[0].blocks)

28

In [28]:
document_blocks = document.pages[0].blocks

In [29]:
document_blocks[0]

bounding_box {
  vertices {
    x: 82
    y: 823
  }
  vertices {
    x: 104
    y: 823
  }
  vertices {
    x: 104
    y: 849
  }
  vertices {
    x: 82
    y: 849
  }
}
paragraphs {
  bounding_box {
    vertices {
      x: 82
      y: 823
    }
    vertices {
      x: 104
      y: 823
    }
    vertices {
      x: 104
      y: 849
    }
    vertices {
      x: 82
      y: 849
    }
  }
  words {
    bounding_box {
      vertices {
        x: 82
        y: 823
      }
      vertices {
        x: 104
        y: 823
      }
      vertices {
        x: 104
        y: 849
      }
      vertices {
        x: 82
        y: 849
      }
    }
    symbols {
      property {
        detected_break {
          type_: LINE_BREAK
        }
      }
      bounding_box {
        vertices {
          x: 82
          y: 823
        }
        vertices {
          x: 104
          y: 823
        }
        vertices {
          x: 104
          y: 849
        }
        vertices {
          x: 82
          

In [30]:
document_blocks[1]

bounding_box {
  vertices {
    x: 183
    y: 129
  }
  vertices {
    x: 695
    y: 116
  }
  vertices {
    x: 698
    y: 229
  }
  vertices {
    x: 186
    y: 242
  }
}
paragraphs {
  bounding_box {
    vertices {
      x: 183
      y: 129
    }
    vertices {
      x: 621
      y: 120
    }
    vertices {
      x: 621
      y: 145
    }
    vertices {
      x: 184
      y: 154
    }
  }
  words {
    property {
      detected_languages {
        language_code: "id"
        confidence: 1.0
      }
    }
    bounding_box {
      vertices {
        x: 183
        y: 129
      }
      vertices {
        x: 204
        y: 129
      }
      vertices {
        x: 205
        y: 154
      }
      vertices {
        x: 184
        y: 154
      }
    }
    symbols {
      bounding_box {
        vertices {
          x: 183
          y: 130
        }
        vertices {
          x: 195
          y: 130
        }
        vertices {
          x: 196
          y: 154
        }
        vertices {